# Collect data

In [1]:
# import Lib
import pandas as pd
import numpy as np
import urllib.robotparser
import time
from bs4 import BeautifulSoup
import json
from pandas.core.frame import DataFrame
import requests
from fractions import Fraction

### Check dữ liệu trên trang này có hợp pháp khi thu thập

In [2]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://store.steampowered.com/robots.txt')
rp.read()

url = 'https://store.steampowered.com/search/?sort_by=Reviews_DESC&filter=topsellers'
rp.can_fetch('*', url)

True

Dữ liệu hợp pháp!

### Crawl data với các thuộc tính

'''  
title  
genre  
tags  
category  
is_dlc  
is_mature  
franchise  
release date  
developer  
publisher  
languages  
overwhelmingly positive count  
user reviews count  
negative count  
price  
'''

In [3]:
Title = []  
Genre = []  
Tags = []   
withDLC = []  
isMature = [] 
Franchise = []
ReleaseDate = []  
Developer = []  
Publisher = []
Languages = []
PositiveReviews = []
TotalReviews = []
NegativeReviews = []  
OriginalPrice = []
DiscountPercent = []

In [4]:
flag_page=True
index=1

# get the games from all the pages and not give it a range because its very dynamic
#while(flag_page):

# Add page parameter
parameter = '&page='+str(index)

# Send a request to the website
web_response = requests.get(url+parameter)
    
# Get the content of the request
web_content = web_response.text

if web_response.status_code == 200: # request successfully
    web_soup = BeautifulSoup(web_content, 'html.parser')
    
    item_link = []
    
#     # Check if the last page
#     if index != 1:
#         # Find the number of buttons on the page and return False if it has 2
#         button = soup.find_all('a',attrs={'class':'pagebtn'})
#         if len(button)!= 2 :
#             flag_page = False

    # Add each link
    links = web_soup.find('div', attrs={'id': 'search_resultsRows'})
    for link in links.find_all('a'):
        item_link.append(link['href'])
                    

        # Price with discount
        with_discount = link.find('div',attrs={'class':'discount_block search_discount_block'})
        # Price without discount
        without_discount = link.find('div',attrs={'class':'discount_block search_discount_block no_discount'})
        # Free price
        free_price = link.find('div',attrs={'class':'discount_block no_discount search_discount_block'})
        
        if with_discount:
            DiscountPercent.append(with_discount.find('div',attrs={'class':'discount_pct'}).text)
            OriginalPrice.append(with_discount.find('div',attrs={'class':'discount_original_price'}).text)      
        elif without_discount:
            DiscountPercent.append(np.nan)
            OriginalPrice.append(without_discount.find('div',attrs={'class':'discount_final_price'}).text)
        elif free_price:
            DiscountPercent.append(np.nan)
            OriginalPrice.append(free_price.find('div',attrs={'class':'discount_final_price free'}).text)
            
#         else:
#             return 

        
    # Add attribute of each game
    for item in item_link:
        response = requests.get(item)
        content = response.text
        if response.status_code == 200:
            soup = BeautifulSoup(content, 'html.parser')
            
            # set all attribute is NaN
            title = np.nan 
            genre = np.nan
            tags = np.nan
            with_dlc = np.nan 
            is_mature = np.nan
            franchise = np.nan
            release_date = np.nan
            developer = np.nan
            publisher = np.nan
            languages = np.nan
            positive_reviews = np.nan
            total_reviews = np.nan
            negative_reviews = np.nan
            
#             # Check if a dlc
#             is_dlc = soup.find('div',attrs={'class':'game_area_bubble game_area_dlc_bubble '})
#             if is_dlc:
#                 return title, genre, tags, with_dlc, is_mature, franchise, release_date, developer, 
#                         publisher, language, positive_reviews, total_reviews, negative_reviews
            
            # Check game has DLC
            try:
                with_dlc_tmp = soup.find('div',attrs={'id':'gameAreaDLCSection'})
                with_dlc = (True if with_dlc_tmp else False) 
            except:
                pass
            
            # Title
            try:
                title = soup.find('div',attrs={'class':'apphub_AppName'}).string.strip() 
            except:
                pass
            
            # Genere
            try:
                genre = [] # A game can have more than one genre
                genre_tmp = soup.find('div',attrs={'id':'genresAndManufacturer'}).find('span')
                for i in genre_tmp.find_all('a'):
                    genre.append(i.text.strip())
            except:
                pass
            
            # Tag
            try:
                tags = [] # A game can have more than one tag
                for i in soup.find('div',attrs={'class':'glance_tags popular_tags'}).find_all('a'):         
                    if i.text.strip() != '+':
                        tags.append(i.text.strip())
            except:
                pass
            
            # Check Mature
            try:
                is_mature_tmp = soup.find('div',attrs={'class':'game_area_content_descriptors'})
                is_mature = (True if is_mature_tmp else False)
            except:
                pass
            
            # Developer, Publisher, Franchise
            try:
                for i in soup.find('div',attrs={'id':'genresAndManufacturer'}).find_all('div',attrs={'class':'dev_row'}):
                    if i.find('b').text == 'Developer:':
                        developer = i.find('a').text
                    elif i.find('b').text == 'Publisher:':
                        publisher = i.find('a').text
                    elif i.find('b').text == 'Franchise:':
                        franchise = i.find('a').text
            except:
                pass
            
            # Release Date
            try:
                release_date = soup.find('div',attrs={'class':'date'}).string.strip()
            except:
                pass 
            
            # Languages
            try:
                languages = [] # A game can have more than one language
                for i in soup.find('div',attrs={'id':'languageTable'}).find_all('td',attrs={'class':'ellipsis'}):   
                    languages.append(i.text.strip())
            except:
                pass
           
            # All review
            try:
                total_reviews = soup.find('label',attrs={'for':'review_type_all'}).find('span',attrs={'class':'user_reviews_count'}).string.strip()
                total_reviews = int(total_reviews.replace(')','').replace('(',''))
            except:
                pass
            
            # Negative review
            try:
                negative_reviews = soup.find('label',attrs={'for':'review_type_negative'}).find('span',attrs={'class':'user_reviews_count'}).string.strip()
                negative_reviews = int(negative_reviews.replace(')','').replace('(',''))
            except:
                pass
            
            # Positive review
            try:
                positive_reviews = soup.find('label',attrs={'for':'review_type_positive'}).find('span',attrs={'class':'user_reviews_count'}).string.strip()
                positive_reviews = int(positive_reviews.replace(')','').replace('(',''))
            except:
                pass
            
            
            Title.append(title)  
            Genre.append(genre)
            Tags.append(tags)  
            withDLC.append(with_dlc)
            isMature.append(is_mature)
            Franchise.append(franchise)
            ReleaseDate.append(release_date) 
            Developer.append(developer)
            Publisher.append(publisher)
            Languages.append(languages)
            PositiveReviews.append(positive_reviews)
            TotalReviews.append(total_reviews)
            NegativeReviews.append(negative_reviews)
    
    
    
# index += 1

In [5]:
print(Title)
print(len(Genre))
print(len(Tags))
print(len(withDLC))
print(len(isMature))
print(len(Franchise))
print(len(ReleaseDate)) 
print(len(Developer))
print(len(Publisher))
print(len(Languages))
print(len(PositiveReviews))
print(len(TotalReviews))
print(len(NegativeReviews))

['Endless Monday: Dreams and Deadlines', 'Senren＊Banka', 'A Short Hike', 'Aseprite', 'Cats Hidden in Paris', "Papa's Freezeria Deluxe", 'TOEM', 'Ib', 'Lil Gator Game', "Patrick's Parabox", 'An Arcade Full of Cats', 'A Castle Full of Cats', 'Dialtown: Phone Dating Sim', 'Frog Detective 3: Corruption at Cowboy County', 'Smushi Come Home', 'Aokana - Four Rhythms Across the Blue - EXTRA2', 'The Upturned', 'Sudocats', "Class of '09: The Re-Up", 'Submachine: Legacy', 'Super Lesbian Animal RPG', 'The Void Rains Upon Her Heart', 'Zodiacats', 'Cattails: Wildwood Story', 'Wallpaper Engine']
25
25
25
25
25
25
25
25
25
25
25
25


In [6]:
len(item_link)

25

In [7]:
data = pd.DataFrame({"Title": Title,
                     "Genre": Genre,
                     "Tags": Tags,
                     "withDLC": withDLC,
                     "isMature": isMature,
                     "Franchise": Franchise,
                     "ReleaseDate": ReleaseDate,
                     "Developer": Developer,
                     "Publisher": Publisher,
                     "Languages": Languages,
                     "PositiveReviews": PositiveReviews,
                     "TotalReviews": TotalReviews,
                     "NegativeReviews": NegativeReviews,
                     "OriginalPrice": OriginalPrice,
                     "DiscountPercent": DiscountPercent})

data

,Title,Genre,Tags,withDLC,isMature,Franchise,ReleaseDate,Developer,Publisher,Languages,PositiveReviews,TotalReviews,NegativeReviews,OriginalPrice,DiscountPercent
0,Endless Monday: Dreams and Deadlines,"[Adventure, Indie]","[Visual Novel, Choose Your Own Adventure, Inte...",False,False,Endless Monday,"4 May, 2023",hcnone,hcnone,"[English, Japanese]",721,721,0,"142.000,00₫",NaN
1,Senren＊Banka,"[Adventure, Casual]","[Visual Novel, Casual, NSFW, Dating Sim, Anime...",False,False,NaN,"14 Feb, 2020",YUZUSOFT,HIKARI FIELD,"[English, Japanese, Simplified Chinese, Tradit...","(16,937)","(17,085)",148,"280.000,00₫",NaN
2,A Short Hike,"[Adventure, Indie]","[Exploration, Adventure, Cute, Relaxing, Indie...",False,False,NaN,"30 Jul, 2019",adamgryu,adamgryu,"[English, French, Spanish - Latin America, Jap...","(14,669)","(14,779)",110,"100.000,00₫",NaN
3,Aseprite,"[Animation & Modeling, Design & Illustration, ...","[Pixel Graphics, Design & Illustration, Animat...",False,False,NaN,"22 Feb, 2016",David Capello,Igara Studio,[English],"(15,453)","(15,580)",127,"188.000,00₫",NaN
4,Cats Hidden in Paris,"[Casual, Indie, Free to Play]","[Cats, Hidden Object, Puzzle, Wholesome, Relax...",True,False,Travellin Cats,"16 Jul, 2023",Travellin Cats,Travellin Cats,"[English, French, Italian, German, Spanish - S...","(11,227)","(11,323)",96,Free,NaN
5,Papa's Freezeria Deluxe,"[Action, Casual, Indie, Simulation, Strategy]","[Cooking, Time Management, Simulation, Casual,...",False,False,Papa Louie,"31 Mar, 2023",Flipline Studios,Flipline Studios,[English],"(6,428)","(6,474)",46,"87.500,00₫",NaN
6,TOEM,"[Adventure, Casual, Indie]","[Cozy, Wholesome, Adventure, Puzzle, Explorati...",True,False,NaN,"17 Sep, 2021",Something We Made,Something We Made,"[English, French, German, Spanish - Spain, Sim...","(4,587)","(4,623)",36,"260.000,00₫",NaN
7,Ib,"[Adventure, Indie]","[Horror, 2D, RPGMaker, Psychological Horror, P...",False,False,PLAYISM,"11 Apr, 2022",kouri,PLAYISM,"[English, Japanese, Simplified Chinese, Tradit...","(3,046)","(3,075)",29,"150.000,00₫",NaN
8,Lil Gator Game,"[Action, Adventure, Casual, Indie]","[Casual, Cozy, Relaxing, Cute, Wholesome, Fami...",False,False,Playtonic Friends,"14 Dec, 2022",MegaWobble,Playtonic Friends,"[English, French, German, Portuguese - Brazil,...","(3,333)","(3,364)",31,"260.000,00₫",NaN
9,Patrick's Parabox,"[Casual, Indie, Strategy]","[Puzzle, Sokoban, Logic, Abstract, Minimalist,...",True,False,NaN,"29 Mar, 2022",Patrick Traynor,Patrick Traynor,"[English, French, Italian, German, Spanish - S...","(2,773)","(2,792)",19,"260.000,00₫",NaN
